In [7]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
import sys
import os

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Base model_V0

In [11]:
%load_ext autoreload
%autoreload 2

nb_dir = os.path.split(os.getcwd())[0]
nb_dir
print(nb_dir)

if nb_dir not in sys.path:
    print("Nb dir is not in system path")
    sys.path.append(nb_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
d:\CodingHenry\food_detection\ml


In [13]:
from classes.model_0 import Cnn_v0